In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("=== CHURN PREDICTION PIPELINE ===")
print("Pipeline para predecir churn de nuevos clientes")

=== CHURN PREDICTION PIPELINE ===
Pipeline para predecir churn de nuevos clientes


In [2]:
# Cargar modelo entrenado
model = joblib.load('../../models/trained/churn_model_xgb.pkl')
label_encoders = joblib.load('../../models/encoders/label_encoders.pkl')

print("✅ Modelo y encoders cargados correctamente")
print(f"Modelo: {type(model).__name__}")
print(f"Encoders disponibles: {list(label_encoders.keys())}")

✅ Modelo y encoders cargados correctamente
Modelo: XGBClassifier
Encoders disponibles: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges', 'tenure_group']


In [7]:
def preprocess_new_customer(customer_data):
    """
    Preprocesa datos de un nuevo cliente para predicción
    """
    # Crear DataFrame
    df_new = pd.DataFrame([customer_data])
    
# 1. Crear tenure_group
    def assign_tenure_group(tenure):
        if tenure < 12:
            return 'Nuevos (<1 año)'
        elif tenure < 24:
            return 'Establecidos (1-2 años)'
        else:
            return 'Veteranos (2+ años)'
    
    df_new['tenure_group'] = df_new['tenure'].apply(assign_tenure_group)
    
    # 2. Crear total_services (usando la misma lógica que antes)
    def count_services(row):
        count = 0
        if row['PhoneService'] == 'Yes':
            count += 1
        if row['MultipleLines'] == 'Yes':
            count += 1
        if row['InternetService'] in ['DSL', 'Fiber optic']:
            count += 1
        if row['OnlineSecurity'] == 'Yes':
            count += 1
        if row['OnlineBackup'] == 'Yes':
            count += 1
        if row['DeviceProtection'] == 'Yes':
            count += 1
        if row['TechSupport'] == 'Yes':
            count += 1
        if row['StreamingTV'] == 'Yes':
            count += 1
        if row['StreamingMovies'] == 'Yes':
            count += 1
        return count
    
    df_new['total_services'] = df_new.apply(count_services, axis=1)
    
    # 3. Quitar customerID
    if 'customerID' in df_new.columns:
        X = df_new.drop(['customerID'], axis=1)
    else:
        X = df_new.copy()
    
    # Identificar categóricas
    categorical_features = X.select_dtypes(include=['object']).columns.tolist()
    
    X_encoded = X.copy()
    
    # Aplicar encoders YA ENTRENADOS (solo transform, NO fit_transform)
    for col in categorical_features:
        if col in label_encoders:
            # Manejar valores no vistos en entrenamiento
            try:
                X_encoded[col] = label_encoders[col].transform(X[col])
            except ValueError:
                print(f"⚠️ Valor desconocido en {col}: {X[col].iloc[0]}")
                # Asignar valor por defecto (ej: 0)
                X_encoded[col] = 0
    
    return X_encoded

In [10]:
def predict_churn(customer_data, threshold=0.4):
    """
    Predice la probabilidad de churn de un cliente
    
    Args:
        customer_data: dict con datos del cliente
        threshold: umbral de decisión (default: 0.4)
    
    Returns:
        dict con predicción, probabilidad y explicación
    """
    # Preprocesar datos
    X_processed = preprocess_new_customer(customer_data)
    
    # Obtener probabilidades
    churn_proba = model.predict_proba(X_processed)[0, 1]  # Probabilidad de churn
    
    # Hacer predicción con threshold
    will_churn = churn_proba >= threshold
    
    # Crear resultado
    result = {
        'customer_id': customer_data.get('customerID', 'Unknown'),
        'churn_probability': float(round(churn_proba, 4)),  # Convertir a float nativo
        'will_churn': bool(will_churn),                     # Convertir a bool nativo
        'risk_level': get_risk_level(churn_proba),
        'confidence': 'High' if abs(churn_proba - 0.5) > 0.3 else 'Medium' if abs(churn_proba - 0.5) > 0.15 else 'Low'
    }
    
    return result

def get_risk_level(probability):
    """Clasifica el riesgo basado en probabilidad"""
    if probability >= 0.7:
        return 'High Risk'
    elif probability >= 0.4:
        return 'Medium Risk'
    else:
        return 'Low Risk'

In [11]:
# Ejemplo de cliente con alta probabilidad de churn
test_customer_high_risk = {
    'customerID': 'TEST001',
    'gender': 'Male',
    'SeniorCitizen': 0,
    'Partner': 'No',
    'Dependents': 'No',
    'tenure': 2,  # Cliente nuevo (alta probabilidad de churn)
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',  # Sin servicios adicionales
    'OnlineBackup': 'No',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'No',
    'StreamingMovies': 'No',
    'Contract': 'Month-to-month',  # ¡El mayor predictor de churn!
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check',  # Método con más churn
    'MonthlyCharges': 85.0,  # Alto para pocos servicios
    'TotalCharges': 170.0
}

# Probar la predicción
print("=== TESTE DE PREDICCIÓN ===")
result = predict_churn(test_customer_high_risk)
print(result)

=== TESTE DE PREDICCIÓN ===
{'customer_id': 'TEST001', 'churn_probability': 0.9103000164031982, 'will_churn': True, 'risk_level': 'High Risk', 'confidence': 'High'}


In [12]:
# Ejemplo de cliente con BAJA probabilidad de churn
test_customer_low_risk = {
    'customerID': 'TEST002',
    'gender': 'Female',
    'SeniorCitizen': 0,
    'Partner': 'Yes',           # Tiene pareja (factor protector)
    'Dependents': 'Yes',        # Tiene dependientes (factor protector)
    'tenure': 36,               # Cliente veterano (baja probabilidad de churn)
    'PhoneService': 'Yes',
    'MultipleLines': 'Yes',     # Múltiples servicios
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'Yes',    # Servicios adicionales de seguridad
    'OnlineBackup': 'Yes',
    'DeviceProtection': 'Yes',
    'TechSupport': 'Yes',       # Soporte técnico
    'StreamingTV': 'Yes',
    'StreamingMovies': 'Yes',
    'Contract': 'Two year',     # ¡Contrato de 2 años! (mayor protección)
    'PaperlessBilling': 'No',   # Billing tradicional (menos churn)
    'PaymentMethod': 'Credit card (automatic)',  # Método con menos churn
    'MonthlyCharges': 95.0,     # Alto pero justificado por muchos servicios
    'TotalCharges': 3420.0      # Cliente de largo plazo
}

# Probar ambos clientes
print("=== CLIENTE ALTO RIESGO ===")
result_high = predict_churn(test_customer_high_risk)
print(result_high)

print("\n=== CLIENTE BAJO RIESGO ===")
result_low = predict_churn(test_customer_low_risk)
print(result_low)

=== CLIENTE ALTO RIESGO ===
{'customer_id': 'TEST001', 'churn_probability': 0.9103000164031982, 'will_churn': True, 'risk_level': 'High Risk', 'confidence': 'High'}

=== CLIENTE BAJO RIESGO ===
{'customer_id': 'TEST002', 'churn_probability': 0.03579999879002571, 'will_churn': False, 'risk_level': 'Low Risk', 'confidence': 'High'}
